# Data Cleaning & Processing
 

The following outlines the steps taken to clean the following data sets:

- Open Toronto Data: TTC Subway, Streetcar, & Bus Delay data sets
- Open Toronto Data: TTC Ridership data
- Canada Historical Weather data

### Libraries

In [24]:
import pandas as pd 
import numpy as np
import os

## Pre-cleaning Data Compilation
### TTC Delay Data
The TTC delay data files have been compiled into their processed forms from the 'data_compiling.py' script in this repository. A few notes regarding the compilation steps taken: 

#### Subway Files

- Files are split by Month and Year of delay record. 
- Data for Jan 2014-April 2017 is stored in a single file, stacked in a single sheet. Therefore, we keep this as is and continue to stack all months of data together for analysis. 
- README and delay codes files moved into a separate 'docs' folder. 

#### Streetcar & Bus Files

- Files are split by Year only, with individual sheets containing each month of data (in comparison to Subway data files). Therefore, slightly modified method of compiling streetcar & bus data was used to first concatenate sheets, then concatenate workbooks.
- Inconsistencies were present in column names for the following: 
    1. Streetcar: Inconsistent columns 'Delay', 'Gap' were merged to more conventional 'Min Delay' and 'Min Gap' columns. 'Incident ID' column removed.
    2. Bus: Inconsistent columns 'Delay'/' Min Delay', 'Gap' were merged to more conventional 'Min Delay' and 'Min Gap' columns 'Incident ID' column removed.

Processed, pre-cleaned files are saved in their respective 'data/processed/ttc' folders.

### TTC Ridership Data
The ridership data is kept in melted format. For analysis purposes, the data is pivoted/unmelted within the 'data_compiling.py' script and is stored in the 'data/processed/ridership' folder.

### Weather Data
No additional compilation is required to prepare the data files for cleaning. These files are simply moved to their respective 'processed' folder.